In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from scripts.load_data import *
from scripts.availabledata import *
from scripts.individual import *
from scripts.cumulative import *
from scripts.bothdatasets import *
from scripts.dataoption import *
from scripts.higher_years import *

In [3]:
configuration = load_configuration("../configuration/configuration.json")

data_individual, data_cumulative, data_student_numbers_first_years, data_student_numbers_higher_years, data_student_numbers_volume, data_latest, data_distances, ensemble_weights = load_data(configuration, StudentYearPrediction.FIRST_YEARS)

dataholder = Cumulative(data_cumulative, data_student_numbers_first_years, configuration)

data_cumulative = dataholder.preprocess()

c:\Users\jjble\Documents\Master\Jaar2_Semester2\Research_Internship\Tijdens\Code\studentprognose\scripts\load_data.py:11: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  data_cumulative = pd.read_csv(paths["path_cumulative"], sep=';', skiprows=[1]) if (paths["path_cumulative"] != "" and os.path.exists(paths["path_cumulative"])) else None
c:\Users\jjble\Documents\Master\Jaar2_Semester2\Research_Internship\Tijdens\Code\studentprognose\scripts\cumulative.py:72: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data[key] = data[key].str.replace('.', '')


In [4]:
predict_year = 2022
predict_week = 15

In [17]:
data_to_predict = data_cumulative[(data_cumulative["Collegejaar"] == predict_year) &
                                               (data_cumulative["Weeknummer"] == predict_week)]

dataholder.prepare_data()

full_data = dataholder.data_cumulative.copy()

full_data = full_data[full_data["Collegejaar"] >= 2016]
full_data = full_data[full_data["Collegejaar"] <= predict_year + 1]

data = transform_data(full_data.copy(), 'ts')

In [43]:
numerus_fixus_list = configuration["numerus_fixus"]

total_mae = 0.0
total_mape = 0.0
count = 0.0

for _, row in data_to_predict.iterrows():
    programme = row['Croho groepeernaam']
    herkomst = row["Herkomst"]
    examtype = row["Examentype"]

    all_years = np.sort(data["Collegejaar"].unique())
    second_year = all_years[1]

    # Train/test split
    if programme not in numerus_fixus_list:
        train = data[(data["Collegejaar"] < predict_year) & (data["Examentype"] == examtype) & (
            ~data['Croho groepeernaam'].isin(numerus_fixus_list))]
        train_copy = data[(data["Collegejaar"] < predict_year + 1) & (data["Examentype"] == examtype) & (
            ~data['Croho groepeernaam'].isin(numerus_fixus_list))]
    elif programme in numerus_fixus_list:
        train = data[(data["Collegejaar"] < predict_year) & (data['Croho groepeernaam'] == programme)]
        train_copy = data[(data["Collegejaar"] < predict_year + 1) & (data['Croho groepeernaam'] == programme)]

    test = data[(data["Collegejaar"] == predict_year) & (data['Croho groepeernaam'] == programme) & (data["Herkomst"] == herkomst)]

    if data_student_numbers_first_years is not None:
        train = train.merge(data_student_numbers_first_years[['Croho groepeernaam', 'Collegejaar', 'Herkomst', 'Aantal_studenten']],
                            on=['Croho groepeernaam', 'Collegejaar', 'Herkomst'])
        train_copy = train_copy.merge(data_student_numbers_first_years[['Croho groepeernaam', 'Collegejaar', 'Herkomst', 'Aantal_studenten']],
                            on=['Croho groepeernaam', 'Collegejaar', 'Herkomst'])

    train = train.drop_duplicates()
    train_copy = train_copy.drop_duplicates()

    # train_copy = train.copy()
    train_copy["Collegejaar"] = train_copy["Collegejaar"] - 1
    train_copy = train_copy.rename(columns={"Aantal_studenten": "Aantal_studenten_next_year"})

    new_train = train.merge(train_copy[['Collegejaar', 'Examentype', 'Faculteit', 'Croho groepeernaam', 'Herkomst', 'Aantal_studenten_next_year']],
                            on=['Collegejaar', 'Examentype', 'Faculteit', 'Croho groepeernaam', 'Herkomst'], how='left')
    # print(new_train.columns)
    # new_train[['Collegejaar', 'Examentype', 'Faculteit', 'Croho groepeernaam', 'Herkomst', 'Aantal_studenten', 'Aantal_studenten_next_year']].to_excel("new_train.xlsx", index=False)

    X_train = new_train.drop(['Aantal_studenten_next_year', "Aantal_studenten"], axis=1)
    # print(new_train["Aantal_studenten_next_year"])
    y_train = new_train.pop('Aantal_studenten_next_year').fillna(0)
    # y_train = y_train.pop('Aantal_studenten')

    # Encode
    # Specify the numeric and categorical column names
    numeric_cols = ['Collegejaar'] + [str(x) for x in get_weeks_list(38)]
    categorical_cols = ['Examentype', 'Faculteit', 'Croho groepeernaam', 'Herkomst']

    # Create transformers for numeric and categorical columns
    numeric_transformer = "passthrough"  # No transformation for numeric columns
    categorical_transformer = OneHotEncoder(handle_unknown='ignore')

    # Create the column transformer
    preprocessor = ColumnTransformer(
        transformers=[
            ('numeric', numeric_transformer, numeric_cols),
            ('categorical', categorical_transformer, categorical_cols)
        ])

    # Apply the preprocessing to the training and test data
    X_train = preprocessor.fit_transform(X_train)
    test = preprocessor.transform(test)
    # Model
    model = XGBRegressor(learning_rate=0.25)

    model.fit(X_train, y_train)

    predictie = model.predict(test)

    final_prediction = int(round(predictie[0], 0))

    nr_of_students = data_student_numbers_first_years[(data_student_numbers_first_years["Collegejaar"] == predict_year + 1) & (data_student_numbers_first_years["Croho groepeernaam"] == programme) & 
                                                      (data_student_numbers_first_years["Herkomst"] == herkomst)]["Aantal_studenten"]
    
    mae = "???"
    mape = "???"
    if len(nr_of_students) > 0:
        mae = abs(nr_of_students.iloc[0] - final_prediction)
        mape = abs((nr_of_students.iloc[0] - final_prediction) / nr_of_students.iloc[0])

        total_mae += mae
        total_mape += mape
        count += 1.0

    print(f"{programme}, {herkomst}, {final_prediction}, MAE: {mae}, MAPE: {mape}")

print(f"Final MAE: {total_mae / count}")
print(f"Final MAPE: {total_mape / count}")

B Algemene Cultuurwetenschappen, EER, 19, MAE: 5, MAPE: 0.35714285714285715
B Algemene Cultuurwetenschappen, NL, 32, MAE: 3, MAPE: 0.10344827586206896
B Algemene Cultuurwetenschappen, Niet-EER, 3, MAE: 1, MAPE: 0.25
B Artificial Intelligence, EER, 30, MAE: 17, MAPE: 0.3617021276595745
B Artificial Intelligence, NL, 108, MAE: 45, MAPE: 0.29411764705882354
B Artificial Intelligence, Niet-EER, 8, MAE: 7, MAPE: 0.4666666666666667
B Artificial Intelligence, onbekend, 7, MAE: ???, MAPE: ???
B Bedrijfskunde, EER, 33, MAE: 4, MAPE: 0.13793103448275862
B Bedrijfskunde, NL, 426, MAE: 45, MAPE: 0.11811023622047244
B Bedrijfskunde, Niet-EER, 8, MAE: 3, MAPE: 0.2727272727272727
B Bestuurskunde, NL, 77, MAE: 6, MAPE: 0.07228915662650602
B Biology, EER, 70, MAE: 22, MAPE: 0.4583333333333333
B Biology, NL, 81, MAE: 9, MAPE: 0.1
B Biology, Niet-EER, 10, MAE: 5, MAPE: 1.0
B Biomedische Wetenschappen, EER, 0, MAE: ???, MAPE: ???
B Biomedische Wetenschappen, NL, 100, MAE: 5, MAPE: 0.05263157894736842
B Bi